In [1]:
import numpy as np, pandas as pd, torch,os
from sentence_transformers import SentenceTransformer
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_hub as hub
import tensorflow_text
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# device = torch.device("cuda:4" if torch.cuda.is_available() else 'cpu')
# BERT_model = SentenceTransformer('distiluse-base-multilingual-cased-v2', device=device)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    tf.config.set_visible_devices(gpus[4], 'GPU')
USE_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3", )

2023-06-01 15:19:57.249715: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 15:19:57.431310: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-01 15:19:58.151410: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-01 15:19:58.151490: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

Instructions for updating:
non-resource variables are not supported in the long term


2023-06-01 15:20:03.108153: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 15:20:04.818329: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21641 MB memory:  -> device: 4, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:21:00.0, compute capability: 8.6


In [2]:
import pickle

In [3]:
data_train = pd.read_csv("./dataset/pre-processing/IntraHD_Sentence_internal-train.csv")
# data_train['label'] = [1 if i=='Y' else 0 for i in data_train.label.values.tolist()]
data_valid = pd.read_csv("./dataset/pre-processing/IntraHD_Sentence_internal-test.csv")
# data_valid['label'] = [1 if i=='Y' else 0 for i in data_valid.label.values.tolist()]
print(data_train.shape, data_valid.shape)

(1011, 2) (113, 2)


In [4]:
print(len(data_train.label.values)-sum(data_train.label.values), sum(data_train.label.values))
print(len(data_valid.label.values)-sum(data_valid.label.values), sum(data_valid.label.values))

683 328
76 37


In [5]:
# pd.set_option('display.max_rows', 2000)
# pd.set_option('display.max_columns', 20)
# data_train[data_train.label==1]

In [6]:
save_path = "/ssd8/chih/project/yadong/predict_baseline_version02_ACM/dataset/IntraHD_pickle_save_05-17"
save_type_folder = ['mUSE_sample1124_sequence_fullcontext_tf']
for save_type_folder_ls in save_type_folder:
    for dp in ['','train','valid','test']:
        os.makedirs(os.path.join(save_path, save_type_folder_ls, dp), exist_ok=True)

In [7]:
USE_training_embeddings=[]
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), 
                 tf.tables_initializer()])
    run_data = data_train['處置其他+症狀處置（描述）'].astype(str).to_list()
    USE_training_embeddings = session.run(USE_model(run_data))
save_folder = save_type_folder[0]
# save_folder = "USE_sample2693_contextsplit"
for idx, (embed_, text_, label_) in enumerate(zip(USE_training_embeddings, data_train['處置其他+症狀處置（描述）'].to_list(),data_train['label'].to_list())):
    with open(os.path.join(save_path, save_folder, "train", "train_USE_internal_pk_data_{}_fixed_re-marked_merge.pkl".format(idx)), "wb") as f:
        save_dict = {"embed": embed_, "text": text_, "label": label_}
        pickle.dump(save_dict, f)

2023-06-01 15:20:06.922625: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21641 MB memory:  -> device: 4, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:21:00.0, compute capability: 8.6
2023-06-01 15:20:07.090834: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-06-01 15:20:16.077261: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [8]:
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), 
                 tf.tables_initializer()])
    run_data = data_valid['處置其他+症狀處置（描述）'].astype(str).to_list()
    USE_validation_embeddings = session.run(USE_model(run_data))
save_folder = save_type_folder[0]
# save_folder = "USE_sample2693_contextsplit"
for idx, (embed_, text_, label_) in enumerate(zip(USE_validation_embeddings, data_valid['處置其他+症狀處置（描述）'].to_list(),data_valid['label'].to_list())):
    with open(os.path.join(save_path, save_folder, "valid", "valid_USE_internal_pk_data_{}_fixed_re-marked_merge.pkl".format(idx)), "wb") as f:
        save_dict = {"embed": embed_, "text": text_, "label": label_}
        pickle.dump(save_dict, f)

2023-06-01 15:20:16.616692: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21641 MB memory:  -> device: 4, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:21:00.0, compute capability: 8.6
